In [ ]:
# Setup assets directory and fetch required images from GitHub
!mkdir -p assets
!wget -O assets/architecture.png https://raw.githubusercontent.com/StuckInTheNet/neural-graph-memory-Work-In-Progress/main/assets/architecture.png
!wget -O assets/dog_caption.jpg https://raw.githubusercontent.com/StuckInTheNet/neural-graph-memory-Work-In-Progress/main/assets/dog_caption.jpg
!wget -O assets/audio_sim_text.png https://raw.githubusercontent.com/StuckInTheNet/neural-graph-memory-Work-In-Progress/main/assets/audio_sim_text.png
!wget -O assets/city_context.jpg https://raw.githubusercontent.com/StuckInTheNet/neural-graph-memory-Work-In-Progress/main/assets/city_context.jpg
!wget -O assets/diagram_analysis.png https://raw.githubusercontent.com/StuckInTheNet/neural-graph-memory-Work-In-Progress/main/assets/diagram_analysis.png


In [ ]:
# Setup assets directory and fetch required images from GitHub
!mkdir -p assets
!wget -O assets/architecture.png https://raw.githubusercontent.com/StuckInTheNet/neural-graph-memory-Work-In-Progress/main/assets/architecture.png
!wget -O assets/dog_caption.jpg https://raw.githubusercontent.com/StuckInTheNet/neural-graph-memory-Work-In-Progress/main/assets/dog_caption.jpg
!wget -O assets/audio_sim_text.png https://raw.githubusercontent.com/StuckInTheNet/neural-graph-memory-Work-In-Progress/main/assets/audio_sim_text.png
!wget -O assets/city_context.jpg https://raw.githubusercontent.com/StuckInTheNet/neural-graph-memory-Work-In-Progress/main/assets/city_context.jpg
!wget -O assets/diagram_analysis.png https://raw.githubusercontent.com/StuckInTheNet/neural-graph-memory-Work-In-Progress/main/assets/diagram_analysis.png


# 🧠 Neural Graph Memory: Full Demo (Colab Ready)
This notebook demonstrates:
- Text & image memory insertion
- Temporal & multimodal graph structure
- Associative retrieval
- Graph traversal
- Persistence (save/load)
- Visualization

In [ ]:
# Uncomment if needed
# !pip install torch torchvision transformers sentence-transformers networkx matplotlib Pillow

In [ ]:
import torch
import torchvision.transforms as T
import torchvision.models as models
from PIL import Image
import networkx as nx
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
import requests
from io import BytesIO
import numpy as np
import pickle

In [ ]:
class MemoryNode:
    def __init__(self, id, embedding, modality, metadata=None):
        self.id = id
        self.embedding = embedding
        self.modality = modality
        self.metadata = metadata or {}

class NeuralGraphMemory:
    def __init__(self):
        self.graph = nx.Graph()
        self.text_encoder = SentenceTransformer('all-MiniLM-L6-v2')
        self.image_encoder = models.resnet18(pretrained=True)
        self.image_encoder.fc = torch.nn.Identity()
        self.image_encoder.eval()
        self.transform = T.Compose([
            T.Resize(256), T.CenterCrop(224),
            T.ToTensor(), T.Normalize(mean=[0.485, 0.456, 0.406],
                                      std=[0.229, 0.224, 0.225])
        ])

    def add_text_node(self, text):
        embedding = self.text_encoder.encode(text)
        return self._add_node(embedding, 'text', {"text": text})

    def add_image_node(self, img_url):
        image = Image.open(BytesIO(requests.get(img_url).content)).convert("RGB")
        tensor = self.transform(image).unsqueeze(0)
        with torch.no_grad():
            embedding = self.image_encoder(tensor).squeeze().numpy()
        return self._add_node(embedding, 'image', {"url": img_url})

    def _add_node(self, embedding, modality, metadata):
        node_id = len(self.graph)
        node = MemoryNode(node_id, embedding, modality, metadata)
        self.graph.add_node(node_id, data=node)
        if node_id > 0:
            self.graph.add_edge(node_id - 1, node_id, relation="temporal")
        return node_id

    def retrieve_nearest(self, query, top_k=3):
        query_emb = self.text_encoder.encode(query)
        similarities = []
        for nid, data in self.graph.nodes(data="data"):
            sim = np.dot(data.embedding, query_emb) / (
                np.linalg.norm(data.embedding) * np.linalg.norm(query_emb))
            similarities.append((nid, sim))
        similarities.sort(key=lambda x: x[1], reverse=True)
        return similarities[:top_k]

    def traverse_temporal(self, start_id=0):
        return list(nx.dfs_preorder_nodes(self.graph, source=start_id))

    def visualize(self):
        labels = nx.get_node_attributes(self.graph, "data")
        pos = nx.spring_layout(self.graph)
        plt.figure(figsize=(10, 6))
        nx.draw(self.graph, pos, with_labels=True, node_color='lightblue')
        node_texts = {k: (v.metadata.get("text") or "img")[:15] for k, v in labels.items()}
        nx.draw_networkx_labels(self.graph, pos, labels=node_texts, font_size=8)
        plt.title("🧠 Neural Graph Memory")
        plt.show()

    def save(self, path='ngm.pkl'):
        with open(path, 'wb') as f:
            pickle.dump(self.graph, f)

    def load(self, path='ngm.pkl'):
        with open(path, 'rb') as f:
            self.graph = pickle.load(f)

## 🔧 Memory Construction

### Adding a Visual Memory Node
We now demonstrate how an AI agent might ingest a visual memory (e.g., a memory architecture diagram) into the graph memory system using CLIP embeddings.

In [ ]:
from PIL import Image
import requests
from io import BytesIO

# Load your architecture image as an example of multimodal input
image_url = 'https://raw.githubusercontent.com/StuckInTheNet/neural-graph-memory/main/assets/architecture.png'
response = requests.get(image_url)
image = Image.open(BytesIO(response.content)).convert('RGB')
image.show()

In [ ]:
ngm.visualize()

## 🔍 Retrieval

In [ ]:
query = "What happened with the dog?"
nearest = ngm.retrieve_nearest(query)
print(f"Query: {query}\n\nTop Matches:")
for nid, score in nearest:
    node = ngm.graph.nodes[nid]["data"]
    print(f" - Node {nid} ({node.modality}): {node.metadata} [Score: {score:.2f}]")

## 🔄 Temporal Traversal

In [ ]:
order = ngm.traverse_temporal()
print("Traversal order:")
for nid in order:
    node = ngm.graph.nodes[nid]["data"]
    print(f" - Node {nid}: {node.metadata}")

## 💾 Persistence

In [ ]:
ngm.save("ngm_demo.pkl")
print("✅ Saved memory graph")

In [ ]:
ngm2 = NeuralGraphMemory()
ngm2.load("ngm_demo.pkl")
print("✅ Loaded memory graph with", len(ngm2.graph), "nodes")

## 🧠 Multimodal Memory Demonstrations
This section simulates how an AI agent stores, embeds, and retrieves rich multimodal memories.
Each example below demonstrates a different modality or modality combination processed into graph memory.

### 🐕 Image + Caption

In [ ]:
from PIL import Image
import requests
from io import BytesIO

image_url = 'https://upload.wikimedia.org/wikipedia/commons/9/91/Golden_Retriever_Carlos_%2810581910556%29.jpg'
response = requests.get(image_url)
image = Image.open(BytesIO(response.content)).convert('RGB')
image.show()

caption = 'Saw a golden retriever chasing a ball in Central Park.'
print(f"Caption: {caption}")

### 🎙️ Text + Audio (Simulated Voice Note)

In [ ]:
# Simulated transcription of a voice note
voice_note = 'Remember to review the memory architecture notes after today’s meeting.'
print(f"Transcribed voice note: {voice_note}")

### 🌆 Image + Timestamp + Context

In [ ]:
image_url = 'https://upload.wikimedia.org/wikipedia/commons/6/63/Times_Square%2C_New_York_City_%28HDR%29.jpg'
response = requests.get(image_url)
image = Image.open(BytesIO(response.content)).convert('RGB')
image.show()

contextual_note = 'Visited Times Square, lots of people, bright screens — 7:45pm, Jan 15'
print(f"Note: {contextual_note}")

### 📊 Diagram + Interpretation

In [ ]:
image_url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/1/10/Transformer.png/600px-Transformer.png'
response = requests.get(image_url)
image = Image.open(BytesIO(response.content)).convert('RGB')
image.show()

interpretation = 'This shows the self-attention mechanism used in GPT-based models.'
print(f"Interpretation: {interpretation}")